In [11]:
 #install fuzzywuzzy
!pip install fuzzywuzzy
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 KB 439.7 kB/s eta 0:00:000:00:010:00:01:01


In [13]:
import pandas as pd
import numpy as np
from pathlib import Path

import fuzzywuzzy as fw
from fuzzywuzzy import process
import openpyxl as op


In [52]:
df1 = pd.read_excel('India_AC_shapefile.xlsx')
df1.head()


,FID,OBJECTID,ST_CODE,ST_NAME,DT_CODE,DIST_NAME,AC_NO,AC_NAME,PC_NO,PC_NAME,PC_ID,STATUS,Shape_Leng,Shape_Area
0,0,1,13,NAGALAND,1,MON,41,Tizit,1,NAGALAND,1301,Pre delimitation,1.381854,0.055845
1,1,1,13,NAGALAND,1,MON,43,Tapi,1,NAGALAND,1301,Pre delimitation,1.056157,0.030387
2,2,1,13,NAGALAND,1,MON,42,Wakching,1,NAGALAND,1301,Pre delimitation,0.980303,0.018828
3,3,1,13,NAGALAND,2,TUENSANG,49,Tamlu,1,NAGALAND,1301,Pre delimitation,1.133296,0.021899
4,4,1,13,NAGALAND,3,MOKOKCHUNG,21,Tuli,1,NAGALAND,1301,Pre delimitation,0.965989,0.022397


In [53]:
df2 = pd.read_excel('goa_masterlist.xlsx')
df2.head()

,year,election,AC no,Sate,AC_NAME,Downloaded,Converted,cleaned,File Status,Error Comment
0,2017,vidhan sabha,1,Goa,Mandrem,1,1,1,file cleaned,NaN
1,2017,vidhan sabha,2,Goa,Pernem,1,1,1,file cleaned,NaN
2,2017,vidhan sabha,3,Goa,Bicholim,1,1,1,file cleaned,NaN
3,2017,vidhan sabha,4,Goa,Tivim,1,1,1,file cleaned,NaN
4,2017,vidhan sabha,5,Goa,Mapusa,1,1,1,file cleaned,NaN


In [57]:
def find_fuzzy_match(name, choices):
    if not name:
        return None
    result = process.extractOne(name, choices)
    if result:
        return result[0]
    else:
        return None

# Create a mapping dictionary for rows in df2
ac_name_mapping = {row['AC_NAME']: row for idx, row in df2.iterrows()}

# Filter df1 for rows with ST_NAME "GOA" and map columns from df2 based on fuzzy matching
df1_goa = df1[df1['ST_NAME'] == 'GOA'].copy()  # Filter rows where ST_NAME is "GOA"
print(df1_goa)
df1_goa['Downloaded'] = df1_goa['AC_NAME'].apply(lambda x: ac_name_mapping[find_fuzzy_match(x, df2['AC_NAME'])]['Downloaded'])
df1_goa['year'] = df1_goa['AC_NAME'].apply(lambda x: ac_name_mapping[find_fuzzy_match(x, df2['AC_NAME'])]['year'])
df1_goa['Converted'] = df1_goa['AC_NAME'].apply(lambda x: ac_name_mapping[find_fuzzy_match(x, df2['AC_NAME'])]['Converted'])
df1_goa['cleaned'] = df1_goa['AC_NAME'].apply(lambda x: ac_name_mapping[find_fuzzy_match(x, df2['AC_NAME'])]['cleaned'])
df1_goa['File Status'] = df1_goa['AC_NAME'].apply(lambda x: ac_name_mapping[find_fuzzy_match(x, df2['AC_NAME'])]['File Status'])
df1_goa['Error Comment'] = df1_goa['AC_NAME'].apply(lambda x: ac_name_mapping[find_fuzzy_match(x, df2['AC_NAME'])]['Error Comment'])

# map df1_goa to df1
df1.loc[df1_goa.index, 'year'] = df1_goa['year']
df1.loc[df1_goa.index, 'Downloaded'] = df1_goa['Downloaded']
df1.loc[df1_goa.index, 'Converted'] = df1_goa['Converted']
df1.loc[df1_goa.index, 'cleaned'] = df1_goa['cleaned']
df1.loc[df1_goa.index, 'File Status'] = df1_goa['File Status']
df1.loc[df1_goa.index, 'Error Comment'] = df1_goa['Error Comment']


# print(df1)
df1.to_excel('India_AC_shapefile_mapped.xlsx', index=False)


       FID  OBJECTID  ST_CODE ST_NAME  DT_CODE  DIST_NAME  AC_NO  \
1728  1728         1       30     GOA        1  NORTH GOA      2   
1729  1729         1       30     GOA        1  NORTH GOA      1   
1730  1730         1       30     GOA        1  NORTH GOA      3   
1731  1731         1       30     GOA        1  NORTH GOA      4   
1732  1732         1       30     GOA        1  NORTH GOA     18   
1733  1733         1       30     GOA        1  NORTH GOA     19   
1734  1734         1       30     GOA        1  NORTH GOA      6   
1735  1735         1       30     GOA        1  NORTH GOA      5   
1736  1736         1       30     GOA        1  NORTH GOA     16   
1737  1737         1       30     GOA        1  NORTH GOA     10   
1738  1738         1       30     GOA        1  NORTH GOA      8   
1739  1739         1       30     GOA        1  NORTH GOA      7   
1740  1740         1       30     GOA        1  NORTH GOA      9   
1741  1741         1       30     GOA        1  

8
4 4 
2 2 4

3 5
2 3 3